In [1]:
import pandas as pd
import numpy as np

import datetime as dt

import matplotlib.pyplot as pls
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
%%time

# Load datasets
# Here, we are loading three different Excel files into separate pandas DataFrames
# Each DataFrame will hold the data from one Excel file

df_1 = pd.read_excel('Wazamba.xlsx')
df_2 = pd.read_excel('Rabona.xlsx')
df_3 = pd.read_excel('Nomin.xlsx')

CPU times: user 5min 21s, sys: 1.69 s, total: 5min 23s
Wall time: 5min 25s


In [4]:
# Concatenate all datasets into one and print first 5 rows, and shape (count of rows and columns)
df = pd.concat([df_1, df_2, df_3])
display(df.drop(['Brand'], axis = 1).head())
df.shape

,License,Created,Processed date,Account ID,Username,Country,"Amount, EUR",Type,Provider,Paysystem,Paymethod,Status
0,Curacao,2023-04-01 00:00:03,2023-04-01 00:00:25,25852603,WZB_MIC77,Germany,20.0,deposit,Impaya,paymentiq,iq_creditcard,complete
1,Curacao,2023-04-01 00:00:03,2023-04-01 00:04:09,27799201,WZB_STELIOS95,ROW2,50.0,deposit,Gate2Way,paymentiq,iq_creditcard,complete
2,Curacao,2023-04-01 00:00:05,2023-04-01 00:00:36,15124274,WZB_ELFIFO,Germany,20.0,deposit,ECommPay,paymentiq,iq_creditcard,complete
3,Curacao,2023-04-01 00:00:19,2023-04-01 02:00:20,25671939,WZB_FIRKI93,Germany,50.0,deposit,NaN,noda,noda,decline
4,Curacao,2023-04-01 00:00:22,2023-04-01 00:00:25,27458097,WZB_ATCS1960,ROW3,50.0,deposit,ECommPay,paymentiq,iq_creditcard,cancel


(2238596, 13)

In [5]:
# Print all unique values in columns Type and Status
print(df['Type'].unique())
print(df['Status'].unique())

['deposit' 'withdraw']
['complete' 'decline' 'cancel' 'pending' 'processing']


In [6]:
# Convert 'Created' column to datetime format and extract the date
# In this step, we are converting the 'Created' column of the DataFrame to datetime format and then extracting 
# only the date component. This is useful for simplifying the data if the time component is not needed
df['Created'] = pd.to_datetime(df['Created']).dt.date
df['Created'] = pd.to_datetime(df['Created'])

# Fill missing values in 'Provider' column with values from 'Paysystem' column
# Here, we are handling missing values in the 'Provider' column by filling them with corresponding values 
# from the 'Paysystem' column
df['Provider'].fillna(df['Paysystem'], inplace=True)

# Drop rows with missing values
# In this step, we are removing any rows that still have missing values after the previous steps
df.dropna(inplace=True)

In [7]:
# Checking the general info regarding our dataframe (count of missing value, format, etc.)
display(df.shape)
print(df.isna().sum())
display(df.info())

(2076433, 13)

Brand             0
License           0
Created           0
Processed date    0
Account ID        0
Username          0
Country           0
Amount, EUR       0
Type              0
Provider          0
Paysystem         0
Paymethod         0
Status            0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2076433 entries, 0 to 513072
Data columns (total 13 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Brand           object        
 1   License         object        
 2   Created         datetime64[ns]
 3   Processed date  datetime64[ns]
 4   Account ID      int64         
 5   Username        object        
 6   Country         object        
 7   Amount, EUR     float64       
 8   Type            object        
 9   Provider        object        
 10  Paysystem       object        
 11  Paymethod       object        
 12  Status          object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(9)
memory usage: 22

None

In [8]:
df = df.reset_index(drop=True)

MOVING_DAYS = 15
df_sliding = df[df['Created'] > (max(df['Created'] - pd.to_timedelta(MOVING_DAYS, unit='d')))]

In [9]:
weights = {
    # 'success_ratio': 0.4,  # The share of successful transactions
    # 'num_transactions': 0.15,  # The total number of transactions
    # 'transaction_amount': 0.15,  # The total amount of transactions
    # 'unique_users': 0.1  # The number of unique users
    
    'success_ratio_dep': 0.4,  # The share of successful transactions
    'success_ratio_wit': 0.4,  # The share of successful transactions
    
    'num_deposits': 0.05,  # The total number of deposits
    'successful_deps': 0.3,  # The total number of successful transactions
    'deposit_amount': 0.05,  # The total amount of deposits
    'share_dep': 0.1,  # The share of all success deps
    
    'num_withdrawals': 0.05,  # The total number of withdrawals
    'successful_wits': 0.3,  # The total number of successful transactions
    'withdrawal_amount': 0.05,  # The total amount of withdrawals
    'share_wit': 0.1,  # The share of all success wits
    
    'unique_users_dep': 0.1,  # The number of unique users
    'unique_users_wit': 0.1  # The number of unique users
}

turned_off_psp_dep = []
turned_off_psp_wit = []

# Avoid unnecessary variable assignments
df = df[df['Status'] != 'init']

In [10]:
df['deposit_amount'] = df.loc[df['Type'] == 'deposit', 'Amount, EUR']
df['withdrawal_amount'] = df.loc[df['Type'] == 'withdraw', 'Amount, EUR']

df_sliding['deposit_amount'] = df_sliding.loc[df_sliding['Type'] == 'deposit', 'Amount, EUR']
df_sliding['withdrawal_amount'] = df_sliding.loc[df_sliding['Type'] == 'withdraw', 'Amount, EUR']

In [11]:
def calculate_stats(group):
    stats = {}
    
    stats['successful_deps'] = ((group['Status'] == 'complete') & (group['Type'] == 'deposit')).sum()
    stats['successful_wits'] = ((group['Status'] == 'complete') & (group['Type'] == 'withdraw')).sum()
    
    stats['num_deposits'] = (group['Type'] == 'deposit').sum()
    stats['deposit_amount'] = group['deposit_amount'].sum()
    
    stats['num_withdrawals'] = (group['Type'] == 'withdraw').sum()
    stats['withdrawal_amount'] = group['withdrawal_amount'].sum()
    
    stats['unique_users_dep'] = group[group['Type'] == 'deposit']['Username'].nunique()
    stats['unique_users_wit'] = group[group['Type'] == 'withdraw']['Username'].nunique()

    return pd.Series(stats)

stats = df.groupby(['License', 'Country', 'Paymethod']).apply(calculate_stats).reset_index()

stats['success_ratio_dep'] = stats['successful_deps'] / stats['num_deposits']
stats['success_ratio_wit'] = stats['successful_wits'] / stats['num_withdrawals']

stats_sliding = df_sliding.groupby(['License', 'Country', 'Paymethod']).apply(calculate_stats).reset_index()

stats_sliding['success_ratio_dep'] = stats_sliding['successful_deps'] / stats_sliding['num_deposits']
stats_sliding['success_ratio_wit'] = stats_sliding['successful_wits'] / stats_sliding['num_withdrawals']

In [12]:
# # stats_t = stats.groupby(['License', 'country'])['num_deposits'].sum().reset_index().rename(columns={'num_deposits': 'total_dep_transactions'})
# stats_t = stats.groupby(['License', 'country']).agg({'successful_deps': 'sum',
#                                                     'successful_wits': 'sum'}).reset_index().rename(columns={'successful_deps': 'total_dep_transactions',
#                                                                                                              'successful_wits': 'total_wit_transactions'})

# stats_sliding_t = stats_sliding.groupby(['License', 'country']).agg({'successful_deps': 'sum',
#                                                                      'successful_wits': 'sum'}).reset_index().rename(columns={'successful_deps': 'total_dep_transactions',
#                                                                                                                               'successful_wits': 'total_wit_transactions'})

In [13]:
def group_and_sum(df, groupby_columns, agg_dict, rename_dict):
    return (
        df.groupby(groupby_columns)
        .agg(agg_dict)
        .reset_index()
        .rename(columns=rename_dict)
    )

# Define aggregation dictionary
agg_dict = {
    'successful_deps': 'sum',
    'successful_wits': 'sum'
}

# Define rename dictionary
rename_dict = {
    'successful_deps': 'total_dep_transactions',
    'successful_wits': 'total_wit_transactions'
}

# Group and aggregate stats
stats_t = group_and_sum(stats, ['License', 'Country'], agg_dict, rename_dict)

# Group and aggregate stats_sliding
stats_sliding_t = group_and_sum(stats_sliding, ['License', 'Country'], agg_dict, rename_dict)

In [14]:
stats = stats.merge(stats_t, on=['License', 'Country'], how='left')
stats_sliding = stats_sliding.merge(stats_sliding_t, on=['License', 'Country'], how='left')

stats['share_dep'] = stats['successful_deps'] / stats['total_dep_transactions']
stats['share_wit'] = stats['successful_wits'] / stats['total_wit_transactions']

stats_sliding['share_dep'] = stats_sliding['successful_deps'] / stats_sliding['total_dep_transactions']
stats_sliding['share_wit'] = stats_sliding['successful_wits'] / stats_sliding['total_wit_transactions']

In [15]:
def process_dataframe(df, condition_column, drop_columns):
    df = df.copy()
    df = df[df[condition_column] != 0]
    df = df.drop(drop_columns, axis=1)
    return df

# Define columns to drop for deposits and withdrawals
drop_columns_dep = ['num_withdrawals', 'withdrawal_amount', 'success_ratio_wit', 'unique_users_wit', 'successful_wits', 'share_wit', 'total_wit_transactions']
drop_columns_wit = ['num_deposits', 'deposit_amount', 'success_ratio_dep', 'unique_users_dep', 'successful_deps', 'share_dep', 'total_dep_transactions']

# Process dataframes
stats_dep = process_dataframe(stats, 'num_deposits', drop_columns_dep)
stats_sliding_dep = process_dataframe(stats_sliding, 'num_deposits', drop_columns_dep)

stats_wit = process_dataframe(stats, 'num_withdrawals', drop_columns_wit)
stats_sliding_wit = process_dataframe(stats_sliding, 'num_withdrawals', drop_columns_wit)

In [16]:
display(stats_sliding_dep.sample(5))
display(stats_wit.sample(5))

,License,Country,Paymethod,successful_deps,num_deposits,deposit_amount,unique_users_dep,success_ratio_dep,total_dep_transactions,share_dep
432,Curacao,Kuwait,iq_muchbetter,5.0,5.0,791.17,2.0,1.000000,1100.0,0.004545
268,Curacao,Germany,noda,2745.0,4149.0,271001.00,855.0,0.661605,49062.0,0.055950
710,Curacao,Slovenia,pgw_iq_creditcard,876.0,1171.0,75883.00,192.0,0.748079,2023.0,0.433020
54,Curacao,Australia,pgw_iq_payid,322.0,414.0,13504.10,79.0,0.777778,9554.0,0.033703
147,Curacao,Canada,pgw_iq_flexepin,1.0,1.0,13.59,1.0,1.000000,5782.0,0.000173


,License,Country,Paymethod,successful_wits,num_withdrawals,withdrawal_amount,unique_users_wit,success_ratio_wit,total_wit_transactions,share_wit
521,Curacao,Iceland,iq_skrill,0.0,3.0,32.00,1.0,0.0,46.0,0.000000
904,Curacao,Norway,pgw_iq_mifinity,4.0,4.0,2702.69,4.0,1.0,998.0,0.004008
181,Curacao,Brazil,beeteller_pix,2.0,5.0,290.58,5.0,0.4,2068.0,0.000967
1367,Curacao,Uzbekistan,cryptopay_ltc,0.0,2.0,25.00,1.0,0.0,0.0,NaN
1159,Curacao,Slovakia,cryptopay_ltc,4.0,8.0,1386.00,2.0,0.5,229.0,0.017467


In [17]:
# Define the columns to group by and the columns to normalize
groupby_columns = ['License', 'Country', 'Paymethod']
columns_to_scale_dep = ['success_ratio_dep', 'num_deposits', 'deposit_amount', 'unique_users_dep', 'successful_deps', 'share_dep'] 
columns_to_scale_wit = ['success_ratio_wit', 'num_withdrawals', 'withdrawal_amount', 'unique_users_wit', 'successful_wits', 'share_wit']

# Group by the given columns and compute the mean
grouped_stats_dep = stats_dep.groupby(groupby_columns)[columns_to_scale_dep].sum().reset_index()
grouped_stats_wit = stats_wit.groupby(groupby_columns)[columns_to_scale_wit].sum().reset_index()

grouped_stats_sliding_dep = stats_sliding_dep.groupby(groupby_columns)[columns_to_scale_dep].sum().reset_index()
grouped_stats_sliding_wit = stats_sliding_wit.groupby(groupby_columns)[columns_to_scale_wit].sum().reset_index()

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply MinMaxScaler to the columns in grouped_stats_dep and grouped_stats_wit
# Append scaled values as new columns
for column in columns_to_scale_dep:
    grouped_stats_dep[column + "_scaled"] = scaler.fit_transform(grouped_stats_dep[[column]])
for column in columns_to_scale_wit:
    grouped_stats_wit[column + "_scaled"] = scaler.fit_transform(grouped_stats_wit[[column]])
    
for column in columns_to_scale_dep:
    grouped_stats_sliding_dep[column + "_scaled"] = scaler.fit_transform(grouped_stats_sliding_dep[[column]])
for column in columns_to_scale_wit:
    grouped_stats_sliding_wit[column + "_scaled"] = scaler.fit_transform(grouped_stats_sliding_wit[[column]])

In [18]:
grouped_stats_dep['weight_dep'] = (
    grouped_stats_dep['success_ratio_dep'] * weights['success_ratio_dep']
    + grouped_stats_dep['num_deposits_scaled'] * weights['num_deposits']
    + grouped_stats_dep['deposit_amount_scaled'] * weights['deposit_amount']
    + grouped_stats_dep['unique_users_dep_scaled'] * weights['unique_users_dep']
    + grouped_stats_dep['successful_deps_scaled'] * weights['successful_deps']
    + grouped_stats_dep['share_dep'] * weights['share_dep']
)

grouped_stats_wit['weight_wit'] = (
    grouped_stats_wit['success_ratio_wit'] * weights['success_ratio_wit']
    + grouped_stats_wit['num_withdrawals_scaled'] * weights['num_withdrawals']
    + grouped_stats_wit['withdrawal_amount_scaled'] * weights['withdrawal_amount']
    + grouped_stats_wit['unique_users_wit_scaled'] * weights['unique_users_wit']
    + grouped_stats_wit['successful_wits_scaled'] * weights['successful_wits']
    + grouped_stats_wit['share_wit'] * weights['share_wit']
)

# Apply log transformation
grouped_stats_dep['weight_dep'] = (grouped_stats_dep['weight_dep']).round(4)
grouped_stats_dep = grouped_stats_dep[~grouped_stats_dep['Paymethod'].isin(turned_off_psp_dep)]

grouped_stats_wit['weight_wit'] = (grouped_stats_wit['weight_wit']).round(4)
grouped_stats_wit = grouped_stats_wit[~grouped_stats_wit['Paymethod'].isin(turned_off_psp_wit)]

# Sort stats by country and weight_dep
grouped_stats_dep = grouped_stats_dep.sort_values(['License', 'Country', 'weight_dep'], ascending=False)
grouped_stats_wit = grouped_stats_wit.sort_values(['License', 'Country', 'weight_wit'], ascending=False)

In [19]:
grouped_stats_sliding_dep['weight_sliding_dep'] = (
    grouped_stats_sliding_dep['success_ratio_dep'] * weights['success_ratio_dep']
    + grouped_stats_sliding_dep['num_deposits_scaled'] * weights['num_deposits'] * 1000
    + grouped_stats_sliding_dep['deposit_amount_scaled'] * weights['deposit_amount'] * 1000
    + grouped_stats_sliding_dep['unique_users_dep_scaled'] * weights['unique_users_dep']
    + grouped_stats_sliding_dep['successful_deps_scaled'] * weights['successful_deps']
    + grouped_stats_sliding_dep['share_dep'] * weights['share_dep']
)

grouped_stats_sliding_wit['weight_sliding_wit'] = (
    grouped_stats_sliding_wit['success_ratio_wit'] * weights['success_ratio_wit']
    + grouped_stats_sliding_wit['num_withdrawals_scaled'] * weights['num_withdrawals'] * 1000
    + grouped_stats_sliding_wit['withdrawal_amount_scaled'] * weights['withdrawal_amount'] * 1000
    + grouped_stats_sliding_wit['unique_users_wit_scaled'] * weights['unique_users_wit']
    + grouped_stats_sliding_wit['successful_wits_scaled'] * weights['successful_wits']
    + grouped_stats_sliding_wit['share_wit'] * weights['share_wit']
)

# Apply log transformation
grouped_stats_sliding_dep['weight_sliding_dep'] = (grouped_stats_sliding_dep['weight_sliding_dep']).round(4)
grouped_stats_sliding_dep = grouped_stats_sliding_dep[~grouped_stats_sliding_dep['Paymethod'].isin(turned_off_psp_dep)]

grouped_stats_sliding_wit['weight_sliding_wit'] = (grouped_stats_sliding_wit['weight_sliding_wit']).round(4)
grouped_stats_sliding_wit = grouped_stats_sliding_wit[~grouped_stats_sliding_wit['Paymethod'].isin(turned_off_psp_wit)]

# Sort stats by country and weight_dep
grouped_stats_sliding_dep = grouped_stats_sliding_dep.sort_values(['License', 'Country', 'weight_sliding_dep'], ascending=False)
grouped_stats_sliding_wit = grouped_stats_sliding_wit.sort_values(['License', 'Country', 'weight_sliding_wit'], ascending=False)

In [20]:
# Merge stats and stats_sliding directly on selected columns
merged_dep = grouped_stats_dep.merge(grouped_stats_sliding_dep, on=['License', 'Country', 'Paymethod'], how='left')
merged_wit = grouped_stats_wit.merge(grouped_stats_sliding_wit, on=['License', 'Country', 'Paymethod'], how='left')

In [21]:
TOTAL_WEIGHT = 0.2
SLIDING_WEIGHT = 0.8
# NAN_WEIGHT = 0.05

def calculate_total_weight(df, weight_col, sliding_weight_col):
    
    # Replace NaN values in sliding_weight_col with 0. In case if there were no transaction during the sliding period
    df[sliding_weight_col].fillna(0, inplace=True)
    
    # Calculate the total weight based on the provided weights
    df['weight'] = df[weight_col] * TOTAL_WEIGHT + df[sliding_weight_col] * SLIDING_WEIGHT
    
    # Replace the weight with weight_col * NAN_WEIGHT if sliding weight is 0
    # df.loc[df[sliding_weight_col] == 0, 'weight'] = df[weight_col] * NAN_WEIGHT
    
    return df

merged_dep = calculate_total_weight(merged_dep, 'weight_dep', 'weight_sliding_dep')
merged_wit = calculate_total_weight(merged_wit, 'weight_wit', 'weight_sliding_wit')

In [22]:
print(merged_dep.shape, merged_wit.shape)

(1223, 30) (583, 30)


In [23]:
display(merged_dep[merged_dep['Country']=='Germany'].sort_values('weight', ascending=False).reset_index())
display(merged_wit[merged_wit['Country']=='Germany'].sort_values('weight', ascending=False).reset_index())

,index,License,Country,Paymethod,success_ratio_dep_x,num_deposits_x,deposit_amount_x,unique_users_dep_x,successful_deps_x,share_dep_x,success_ratio_dep_scaled_x,num_deposits_scaled_x,deposit_amount_scaled_x,unique_users_dep_scaled_x,successful_deps_scaled_x,share_dep_scaled_x,weight_dep,success_ratio_dep_y,num_deposits_y,deposit_amount_y,unique_users_dep_y,successful_deps_y,share_dep_y,success_ratio_dep_scaled_y,num_deposits_scaled_y,deposit_amount_scaled_y,unique_users_dep_scaled_y,successful_deps_scaled_y,share_dep_scaled_y,weight_sliding_dep,weight
0,806,Curacao,Germany,powercash_giropay,1.000000,119973.0,6626771.15,4796.0,119973.0,0.292889,1.000000,0.478527,4.365780e-01,0.315481,0.676800,0.292889,0.7096,1.000000,13392.0,800799.00,1484.0,13392.0,0.272961,1.000000,0.470322,0.505161,0.405524,0.610169,0.272961,49.4250,39.68192
1,807,Curacao,Germany,gatetwoway_sofort,0.999759,78999.0,3957357.16,3871.0,78980.0,0.192813,0.999759,0.315096,2.607144e-01,0.254622,0.445548,0.192813,0.6071,0.998194,10520.0,503148.49,1297.0,10501.0,0.214035,0.998194,0.369451,0.317395,0.354389,0.478449,0.214035,34.9420,28.07502
2,814,Curacao,Germany,pgw_iq_creditcard,0.765349,8502.0,608778.00,1205.0,6507.0,0.015885,0.765349,0.033908,4.010658e-02,0.079216,0.036708,0.015885,0.3304,0.765349,8502.0,608778.00,1205.0,6507.0,0.132628,0.765349,0.298574,0.384029,0.329232,0.296473,0.132628,34.5714,27.72320
3,809,Curacao,Germany,noda_de,0.678052,84116.0,5015471.69,8594.0,57035.0,0.139239,0.678052,0.335506,3.304240e-01,0.565366,0.321750,0.139239,0.4715,0.692849,8950.0,520463.00,1448.0,6201.0,0.126391,0.692849,0.314309,0.328318,0.395680,0.282531,0.126391,32.5454,26.13062
4,813,Curacao,Germany,noda,0.662702,51708.0,3447225.00,6312.0,34267.0,0.083656,0.662702,0.206241,2.271063e-01,0.415225,0.193309,0.083656,0.3946,0.661605,4149.0,271001.00,855.0,2745.0,0.055950,0.661605,0.145687,0.170951,0.233525,0.125068,0.055950,16.1630,13.00932
5,808,Curacao,Germany,iq_creditcard,0.711669,109610.0,8542632.43,5126.0,78006.0,0.190435,0.711669,0.437193,5.627969e-01,0.337193,0.440053,0.190435,0.5194,0.779729,3986.0,267494.86,676.0,3108.0,0.063348,0.779729,0.139962,0.168739,0.184578,0.141607,0.063348,15.8142,12.75524
6,822,Curacao,Germany,pgw_iq_sofort,0.709034,1605.0,187376.00,244.0,1138.0,0.002778,0.709034,0.006398,1.234419e-02,0.015988,0.006420,0.002778,0.2884,0.709034,1605.0,187376.00,244.0,1138.0,0.023195,0.709034,0.056336,0.118198,0.066448,0.051850,0.023195,9.0349,7.28560
7,823,Curacao,Germany,pgw_iq_applepay,0.689013,2148.0,126500.00,422.0,1480.0,0.003613,0.689013,0.008564,8.333619e-03,0.027699,0.008349,0.003613,0.2821,0.689013,2148.0,126500.00,422.0,1480.0,0.030166,0.689013,0.075407,0.079796,0.115122,0.067432,0.030166,8.0705,6.51282
8,820,Curacao,Germany,iq_webredirect,0.653812,14599.0,708183.79,2024.0,9545.0,0.023302,0.653812,0.058226,4.665553e-02,0.133101,0.053846,0.023302,0.2986,0.699690,969.0,40121.06,217.0,678.0,0.013819,0.699690,0.033998,0.025306,0.059065,0.030891,0.013819,3.2617,2.66908
9,815,Curacao,Germany,pgw_iq_funanga,0.806667,900.0,39110.00,223.0,726.0,0.001772,0.806667,0.003586,2.576277e-03,0.014606,0.004096,0.001772,0.3258,0.806667,900.0,39110.00,223.0,726.0,0.014798,0.806667,0.031575,0.024668,0.060705,0.033078,0.014798,3.1523,2.58700


,index,License,Country,Paymethod,success_ratio_wit_x,num_withdrawals_x,withdrawal_amount_x,unique_users_wit_x,successful_wits_x,share_wit_x,success_ratio_wit_scaled_x,num_withdrawals_scaled_x,withdrawal_amount_scaled_x,unique_users_wit_scaled_x,successful_wits_scaled_x,share_wit_scaled_x,weight_wit,success_ratio_wit_y,num_withdrawals_y,withdrawal_amount_y,unique_users_wit_y,successful_wits_y,share_wit_y,success_ratio_wit_scaled_y,num_withdrawals_scaled_y,withdrawal_amount_scaled_y,unique_users_wit_scaled_y,successful_wits_scaled_y,share_wit_scaled_y,weight_sliding_wit,weight
0,388,Curacao,Germany,pgw_iq_bankintl,0.449719,11565.0,3716574.15,1852.0,5201.0,0.259790,0.449719,0.395418,0.345640,0.415581,0.428560,0.259790,0.4130,0.530277,3468.0,1092061.15,1021.0,1839.0,0.633919,0.530277,1.000000,1.000000,1.000000,1.000000,0.633919,100.6755,80.62300
1,387,Curacao,Germany,iq_bankintl,0.402859,29246.0,10752716.65,4455.0,11782.0,0.588511,0.402859,1.000000,1.000000,1.000000,0.970831,0.588511,0.7112,0.535770,1286.0,405066.01,507.0,689.0,0.237504,0.535770,0.370637,0.370915,0.496078,0.374660,0.237504,37.4777,30.12440
2,405,Curacao,Germany,pgw_iq_creditcard,0.256360,511.0,157793.00,188.0,131.0,0.006543,0.256360,0.017439,0.014674,0.041985,0.010794,0.006543,0.1122,0.256360,511.0,157793.00,188.0,131.0,0.045157,0.256360,0.147101,0.144486,0.183333,0.071234,0.045157,14.7261,11.80332
3,399,Curacao,Germany,iq_creditcard,0.308747,4230.0,1573440.17,891.0,1306.0,0.065235,0.308747,0.144606,0.146329,0.199820,0.107614,0.065235,0.1968,0.349593,246.0,71362.20,104.0,86.0,0.029645,0.349593,0.070666,0.065341,0.100980,0.046765,0.029645,6.9673,5.61320
4,406,Curacao,Germany,pgw_iq_bankdomestic,0.231330,549.0,164674.00,300.0,127.0,0.006344,0.231330,0.018738,0.015314,0.067131,0.010465,0.006344,0.1047,0.293750,160.0,49928.00,116.0,47.0,0.016201,0.293750,0.045861,0.045714,0.112745,0.025557,0.016201,4.7168,3.79438
5,400,Curacao,Germany,iq_bankdomestic,0.360960,2208.0,670467.53,742.0,797.0,0.039810,0.360960,0.075466,0.062353,0.166367,0.065672,0.039810,0.1916,0.711538,52.0,19570.00,26.0,37.0,0.012754,0.711538,0.014710,0.017915,0.024510,0.020120,0.012754,1.9256,1.57880
6,403,Curacao,Germany,pgw_iq_jeton,0.333333,51.0,22051.00,10.0,17.0,0.000849,0.333333,0.001710,0.002050,0.002021,0.001401,0.000849,0.1342,0.333333,51.0,22051.00,10.0,17.0,0.005860,0.333333,0.014422,0.020186,0.008824,0.009244,0.005860,1.8680,1.52124
7,392,Curacao,Germany,cryptopay_btc,0.766055,218.0,61307.00,120.0,167.0,0.008342,0.766055,0.007420,0.005701,0.026718,0.013761,0.008342,0.3147,0.941176,17.0,3172.00,13.0,16.0,0.005515,0.941176,0.004615,0.002899,0.011765,0.008700,0.005515,0.7565,0.66814
8,390,Curacao,Germany,wc_creditcard,0.900000,10.0,4800.00,1.0,9.0,0.000450,0.900000,0.000308,0.000446,0.000000,0.000742,0.000450,0.3603,0.900000,10.0,4800.00,1.0,9.0,0.003102,0.900000,0.002596,0.004390,0.000000,0.004894,0.003102,0.7111,0.64094
9,401,Curacao,Germany,pgw_iq_mifinity,0.466667,15.0,5785.00,7.0,7.0,0.000350,0.466667,0.000479,0.000538,0.001347,0.000577,0.000350,0.1871,0.466667,15.0,5785.00,7.0,7.0,0.002413,0.466667,0.004038,0.005292,0.005882,0.003806,0.002413,0.6551,0.56150


In [24]:
merged_dep.to_csv('merged_dep.csv')
merged_wit.to_csv('merged_wit.csv')